In [1]:
import os
import sys
import pandas as pd

from run_cebab import get_cbm_standard, get_cbm_joint

/Volumes/Satechi/Applications/miniconda3/envs/cbm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# can only run once when kernal start
os.chdir('run_cebab')

In [3]:
lr_rate_dt = {
    'lstm': 1e-2,
    'gpt2': 1e-4,
    'roberta-base': 1e-5,
    'bert-base-uncased': 1e-5
}

def get_average_scores(score_list):
    if not score_list:
        return (0.0, 0.0)

    s1 = s2 = 0.0
    n = 0
    for a, b in score_list:
        s1 += a
        s2 += b
        n += 1
    return ((s1 / n * 100), (s2 / n * 100))

def get_tuple_2f_fmt(tp):
    f1, f2 = tp
    return f"{f1:.2f}/{f2:.2f}"

In [ ]:
num_epochs = 20

plms_funcs = {
    'PLMs': get_cbm_standard,
    'CBE-PLMs': get_cbm_joint
}

data_types = ['pure_cebab', 'aug_cebab']
model_names = ['bert-base-uncased', 'roberta-base', 'gpt2', 'lstm']
plms_results = {
    'data_type': [],
    'function': [],
    'model': [],
    'score': []
}

# functions
for f_name, f in plms_funcs.items():
    print(f"Running {f_name}...")
    for data_type in data_types:
        print(f"\tRunning {data_type}...")
        for model_name in model_names:
            lr = lr_rate_dt.get(model_name)
            print(f"\t\tRunning {model_name}... with learning rate: {lr}")
            plms_results['data_type'].append(data_type)
            plms_results['function'].append(f_name)
            plms_results['model'].append(model_name)
            plms_results['score'].append(
                f(
                    model_name=model_name,
                    num_epochs=num_epochs,
                    data_type=data_type,
                    max_len=512,
                    batch_size=8,
                    optimizer_lr=lr
                )
            )

Running standard...
	Running pure_cebab...
		Running bert-base-uncased... with learning rate: 1e-05


Val: 100%|██████████| 210/210 [00:11<00:00, 18.61batch/s]


Epoch 1: Val Acc = 55.05080693365212 Val Macro F1 = 63.279823052991034


Val: 100%|██████████| 210/210 [00:11<00:00, 18.61batch/s]


Epoch 2: Val Acc = 60.848774656306034 Val Macro F1 = 68.88938213437918


...

Epoch 20: Val Acc = 66.46742378959952 Val Macro F1 = 77.98465375384265
Test!


Test: 100%|██████████| 211/211 [00:11<00:00, 18.60batch/s]


Epoch 1: Test Acc = 68.96142433234421 Test Macro F1 = 79.1050569676012
		Running roberta-base... with learning rate: 1e-05


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Val: 100%|██████████| 210/210 [00:11<00:00, 18.88batch/s]


Epoch 1: Val Acc = 66.94560669456067 Val Macro F1 = 75.42614949862093


In [4]:
df = pd.DataFrame.from_dict(plms_results)
df['score_avg'] = df.score.apply(get_average_scores)
df['score_fmted'] = df.score_avg.apply(get_tuple_2f_fmt)
df

In [ ]:
df.to_csv("std.csv", index=False)

In [5]:
df = pd.read_csv('std.csv')
df.pivot(index=['model'], columns=['data_type'], values='score_fmted')

data_type,aug_cebab,pure_cebab
model,,
bert-base-uncased,68.96/79.11,67.44/78.64
gpt2,64.99/76.25,63.41/76.81
lstm,40.00/61.78,45.23/65.12
roberta-base,69.38/80.66,71.70/81.28


# Debug lstm

In [ ]:
# get_cbm_standard(
#     model_name='lstm',
#     data_type='aug_cebab',
#     max_len=512,
#     batch_size=8,
#     optimizer_lr=1e-2,
#     num_epochs=20,
# )

In [ ]:
# get_cbm_standard(
#     model_name='lstm',
#     data_type='pure_cebab',
#     max_len=512,
#     batch_size=8,
#     optimizer_lr=1e-2,
#     num_epochs=20,
# )